In [242]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [179]:
import networkx as nx
import time
from random import randint
from psutil import Popen
from networkx import Graph
from tempfile import NamedTemporaryFile

This is how to generate graphs:
- how to add random weights: https://stackoverflow.com/questions/31804117/how-to-create-a-random-networkx-graph-with-random-weights

In [234]:
n_vertices = 5000
g = nx.gnp_random_graph(n_vertices, 0.1, 42)
for u, v in g.edges():
    g.edges[u, v]["weight"] = randint(1, n_vertices)

In [235]:
def trial(graph: Graph, graph_adj: str, p_queue: str, start: int, end: int, program: str = "./a.out") -> dict[str, float]:
    """Perform one trial of djistrka shortest path.

    Args:
        graph: Weighted graph to perform shortest path on.
            Weights should be set on node data with the "weight" key
        graph_adj: Graph adjcency implementation to use. Either 'matrix' or 'list'.
        p_queue: Priority Queue implementation to use. Either 'array' or 'heap'.
        start: Starting node to traverse from.
        end: End node to traverse to.
        program: Path to the executable compiled from code.cpp.
    """
    with NamedTemporaryFile("w") as f:
        # encode trial parameters into input file
        f.write(f"{graph.number_of_nodes()} {start} {end}\n")
        for src, dest, data in graph.edges(data=True):
            # simulate undirected edge via 2 directed edges in opposite directions
            f.write(f"{src} {data['weight']} {dest}\n")
            f.write(f"{dest} {data['weight']} {src}\n")
        f.flush()

        # run program as a subprocess
        process = Popen([program, graph_adj, p_queue, f.name])
        stats = {"ram": 0}
        while True:
            # record system resource usage
            cpu =  process.cpu_times()
            stats["cpu_time"] = cpu.user + cpu.system
            stats["ram"] = max(stats["ram"], process.memory_info().vms)
            time.sleep(1e-6)
            
            if process.poll() is not None:
                break
        return stats
        
    

In [240]:
trial(g, "matrix", "heap", 0, 5)

shortest: 0 416 1088 4649 3719 4926 136 2384 5 


{'ram': 186896384, 'cpu_time': 0.48}